In [1]:
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as F

# Hotfix
!cp /runtime-addons/cmladdon-2.0.40-b150/log4j.properties /etc/spark/conf/

# SparkSession creation logic (your existing code)
spark = SparkSession\
    .builder\
    .appName("project_aiu")\
    .config("spark.hadoop.fs.azure.ext.cab.required.group","eur-app-aiu-dev")\
    .config("spark.kerberos.access.hadoopFileSystems","abfs://storage-fs@cdpdldev0.dfs.core.windows.net/")\
    .config("spark.driver.cores","1")\
    .config("spark.driver.memory","8G")\
    .config("spark.executor.memory","10G")\
    .config("spark.executor.cores","1")\
    .config("spark.executor.instances","2")\
    .config("spark.dynamicAllocation.maxExecutors", "5")\
    .enableHiveSupport() \
    .getOrCreate()

# Read data
df = spark.sql(f"SELECT * FROM `project_aiu`.`osn_ec_datadump_bucketed_full` LIMIT 10;")

# Save the original column names for later
original_columns = df.columns

# Convert 'event_time' to datetime
df = df.withColumn("event_time", F.to_timestamp("event_time"))

# Extract the year and month from event_time to create a 'month_id'
df = df.withColumn("month_id", F.date_format("event_time", "Y-MM"))

df_p = df.toPandas()

Setting spark.hadoop.yarn.resourcemanager.principal to quinten.goens
23/09/15 08:17:22 737 ERROR TSaslTransport: SASL negotiation failure
javax.security.sasl.SaslException: GSS initiate failed [Caused by GSSException: No valid credentials provided (Mechanism level: Server not found in Kerberos database (7) - LOOKING_UP_SERVER)]
	at com.sun.security.sasl.gsskerb.GssKrb5Client.evaluateChallenge(GssKrb5Client.java:211)
	at org.apache.thrift.transport.TSaslClientTransport.handleSaslStartMessage(TSaslClientTransport.java:95)
	at org.apache.thrift.transport.TSaslTransport.open(TSaslTransport.java:265)
	at org.apache.thrift.transport.TSaslClientTransport.open(TSaslClientTransport.java:38)
	at org.apache.hadoop.hive.metastore.security.TUGIAssumingTransport$1.run(TUGIAssumingTransport.java:51)
	at org.apache.hadoop.hive.metastore.security.TUGIAssumingTransport$1.run(TUGIAssumingTransport.java:48)
	at java.security.AccessController.doPrivileged(Native Method)
	at javax.security.auth.Subject.doAs

Py4JError: An error occurred while calling o64.sql